In [4]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
import matplotlib.pyplot as plt
import pandas as pd

import os
import glob

In [2]:
dataloc = pd.read_csv('C:/spaceweather/data/eit/eit.csv', names=['date','eit','lasco'])
dataloc

,date,eit,lasco
0,1996/5/1,C:/spaceweather/data/EIT/1996/5/1/01_01_48.fts,C:/spaceweather/data/LASCO/1996/5/1/03_55_04.fts
1,1996/5/1,C:/spaceweather/data/EIT/1996/5/1/09_54_10.fts,C:/spaceweather/data/LASCO/1996/5/1/08_41_46.fts
2,1996/5/1,C:/spaceweather/data/EIT/1996/5/1/18_32_56.fts,C:/spaceweather/data/LASCO/1996/5/1/17_20_34.fts
3,1996/5/2,C:/spaceweather/data/EIT/1996/5/2/02_39_52.fts,C:/spaceweather/data/LASCO/1996/5/2/01_11_29.fts
4,1996/5/2,C:/spaceweather/data/EIT/1996/5/2/11_20_06.fts,C:/spaceweather/data/LASCO/1996/5/2/09_51_41.fts
...,...,...,...
16674,2008/12/29,C:/spaceweather/data/EIT/2008/12/29/19_00_15.fts,C:/spaceweather/data/LASCO/2008/12/29/19_31_44...
16675,2008/12/30,C:/spaceweather/data/EIT/2008/12/30/01_00_13.fts,C:/spaceweather/data/LASCO/2008/12/30/01_31_39...
16676,2008/12/30,C:/spaceweather/data/EIT/2008/12/30/07_00_15.fts,C:/spaceweather/data/LASCO/2008/12/30/07_31_39...
16677,2008/12/30,C:/spaceweather/data/EIT/2008/12/30/13_00_13.fts,C:/spaceweather/data/LASCO/2008/12/30/13_31_38...


In [14]:
dataset = dataloc.values
eit = dataset[:,1]
lasco = dataset[:,2]


['C:/spaceweather/data/EIT/1996/5/1/01_01_48.fts'
 'C:/spaceweather/data/EIT/1996/5/1/09_54_10.fts'
 'C:/spaceweather/data/EIT/1996/5/1/18_32_56.fts' ...
 'C:/spaceweather/data/EIT/2008/12/30/07_00_15.fts'
 'C:/spaceweather/data/EIT/2008/12/30/13_00_13.fts'
 'C:/spaceweather/data/EIT/2008/12/30/19_00_13.fts']
['C:/spaceweather/data/LASCO/1996/5/1/03_55_04.fts'
 'C:/spaceweather/data/LASCO/1996/5/1/08_41_46.fts'
 'C:/spaceweather/data/LASCO/1996/5/1/17_20_34.fts' ...
 'C:/spaceweather/data/LASCO/2008/12/30/07_31_39.fts'
 'C:/spaceweather/data/LASCO/2008/12/30/13_31_38.fts'
 'C:/spaceweather/data/LASCO/2008/12/30/18_06_04.fts']


In [19]:
#if not os.path.exists("./gan_images"):
#    os.makedirs("./gan_images")

                              
#생성자 모델
generator = Sequential()
#노드 128
generator.add(Dense(128*4*4, input_dim=100, activation=LeakyReLU(0.2)))
#256 만들어야함 upsampling 2번(x4), input_dim은 100차원 벡터, 입력층에서 LeakyReLU
generator.add(BatchNormalization())
generator.add(Reshape((4, 4, 128)))
generator.add(UpSampling2D())
#업샘플링(가로세로x2)
generator.add(Conv2D(64, kernel_size=3, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
#업샘플링(가로세로x2)
generator.add(Conv2D(1, kernel_size=3, padding='same', activation='tanh'))
#출력층에서 tanh

#판별자 모델
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=3, strides=2, input_shape=(16,16,1), padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
#노드 30% 꺼줌 dropout
discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
#노드 30% 꺼줌 dropout
discriminator.add(Flatten())
#2차원 1차원으로
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
#참 거짓 0~1로 구분
discriminator.trainable = False

#생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

#신경망을 실행시키는 함수를 만듭니다.
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _) = 
    X_train = X_train.reshape(X_train.shape[0], 16, 16, 1).astype('float32')
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for i in range(epoch):
        # 실제 데이터 판별자 입력
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)

        #가상 이미지 판별자 입력
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)

        #판별자와 생성자의 오차 계산
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)

        print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

        #중간 과정을 이미지로 저장해 주는 부분입니다
        if i % saving_interval == 0:
            #r, c = 5, 5
            noise = np.random.normal(0, 1, (25, 100))
            gen_imgs = generator.predict(noise)

            # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5

            fig, axs = plt.subplots(5, 5)
            count = 0
            for j in range(5):
                for k in range(5):
                    axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                    axs[j, k].axis('off')
                    count += 1
            fig.savefig("gan_images/gan%d.png" % i)

gan_train(4001, 32, 200)  #4000번 반복, 배치 사이즈  32,  200번 마다 결과가 저장


'                               \n#생성자 모델을 만듭니다.\ngenerator = Sequential()\n#노드 128\ngenerator.add(Dense(128*4*4, input_dim=100, activation=LeakyReLU(0.2)))\n#256 만들어야함 upsampling 2번(x4), input_dim은 100차원 벡터, 입력층에서 LeakyReLU\ngenerator.add(BatchNormalization())\ngenerator.add(Reshape((4, 4, 128)))\ngenerator.add(UpSampling2D())\n#업샘플링(가로세로x2)\ngenerator.add(Conv2D(64, kernel_size=3, padding=\'same\'))\ngenerator.add(BatchNormalization())\ngenerator.add(Activation(LeakyReLU(0.2)))\ngenerator.add(UpSampling2D())\n#업샘플링(가로세로x2)\ngenerator.add(Conv2D(1, kernel_size=3, padding=\'same\', activation=\'tanh\'))\n#출력층에서 tanh\n\n#판별자 모델을 만듭니다.\ndiscriminator = Sequential()\ndiscriminator.add(Conv2D(64, kernel_size=3, strides=2, input_shape=(16,16,1), padding="same"))\ndiscriminator.add(Activation(LeakyReLU(0.2)))\ndiscriminator.add(Dropout(0.3))\n#노드 30% 꺼줌\ndiscriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))\ndiscriminator.add(Activation(LeakyReLU(0.2)))\ndiscriminator.add(